# TFIDF + multinomial bayes

In [13]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# import CountVectorizer, nltk
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [23]:
# loading all files. 

data = pd.read_csv("all-data.csv", encoding='unicode_escape',names=['Sentiment', 'Text'])
data['Sentiment']

# Split data into training and test sets
sentence_train, sentence_test, y_train, y_test =  train_test_split(data["Text"], data["Sentiment"], test_size = 0.20, shuffle = False)
print(sentence_train)
print("-------------")
print(y_train)


0       According to Gran , the company has no plans t...
1       Technopolis plans to develop in stages an area...
2       The international electronic industry company ...
3       With the new production plant the company woul...
4       According to the company 's updated strategy f...
                              ...                        
3871    The newspaper 's best sales asset is high qual...
3872    The non-recurring costs caused to Talentum 's ...
3873    The ongoing project where Tekla Structures is ...
3874    The operations to be sold include manufacturin...
3875    The options might include a partial or total d...
Name: Text, Length: 3876, dtype: object
-------------
0        neutral
1        neutral
2       negative
3       positive
4       positive
          ...   
3871     neutral
3872     neutral
3873     neutral
3874     neutral
3875     neutral
Name: Sentiment, Length: 3876, dtype: object


In [16]:
# initialize CountVectorizer
#movieVzer= CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize, max_features=3000) # use top 3000 words only. 78.25% acc.
countVectorizer = CountVectorizer(min_df=2)         # use all 25K words. Higher accuracy

# fit and tranform using training text 
sentence_train_counts = countVectorizer.fit_transform(sentence_train)

# Convert raw frequency counts into TF-IDF values
tfidfTransformer = TfidfTransformer()
sentence_train_tfidf = tfidfTransformer.fit_transform(sentence_train_counts)

# Using the fitted vectorizer and transformer, tranform the test data
sentence_test_counts = countVectorizer.transform(sentence_test)
sentence_test_tfidf = tfidfTransformer.transform(sentence_test_counts)

In [21]:
# Now ready to build a classifier. 
# We will use Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB

# Train a Multimoda Naive Bayes classifier. Again, we call it "fitting"
clf = MultinomialNB()
clf.fit(sentence_train_tfidf, y_train)

MultinomialNB()

In [22]:
predicted = clf.predict(sentence_test_tfidf)
print(np.mean(predicted == y_test))

0.41855670103092785
